RAIN PREDICTION

Predict next day rain in Thai ...

    Predicting next day-rain using a dataset containing years of daily weather
    
Content

    1. IMPORTING LIBRARIES
    2. LOADING DATA
    3. DATA VISUALIZATION AND CLEANINGS
    4. DATA PREPROCESSING
    5. MODEL BUILDING
    6. CONCLUSION
    7. END

LIBRARIES

IMPORTING LIBRARIES

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from keras import callbacks

ModuleNotFoundError: No module named 'tensorflow'